<a href="https://colab.research.google.com/github/MuhammadAbuSufian/eShop/blob/main/I2IQA_PickScore_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
from transformers import AutoProcessor, AutoModel
from PIL import Image
import torch
import requests
from io import BytesIO


In [10]:
# Setup the device for model computation
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model and processor from Hugging Face
processor_name_or_path = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
model_pretrained_name_or_path = "yuvalkirstain/PickScore_v1"
processor = AutoProcessor.from_pretrained(processor_name_or_path)
model = AutoModel.from_pretrained(model_pretrained_name_or_path).eval().to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
# Function to calculate the alignment score
def calc_alignment_score(prompt, image):
    # Preprocess the image
    image_input = processor(images=image, return_tensors="pt").to(device)

    # Preprocess the text
    text_input = processor(text=prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        # Get embeddings
        image_emb = model.get_image_features(**image_input)
        image_emb = image_emb / image_emb.norm(dim=-1, keepdim=True)

        text_emb = model.get_text_features(**text_input)
        text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)

        # Compute the dot product of the embeddings (cosine similarity)
        match_score = (text_emb @ image_emb.T).squeeze()

        # Scale the match score to range from 0 to 1
        scaled_match_score = (match_score + 1) / 2

    return scaled_match_score.item()

In [12]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Load the CSV file
raw_data = pd.read_csv('/content/drive/MyDrive/Colab/I2IQA/I2IQA_annotation.csv')
data = raw_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import os
# Initialize an empty list to store results
results = []
expected_results = []

# Iterate over each row in the dataframe
for index, row in data.iterrows():
    image_name = row['Generated_image']
    prompt = row['Text_prompt']
    human_feedback_score = row['MOS_a']

    # Construct the path to the image file
    image_path = os.path.join('/content/drive/MyDrive/Colab/I2IQA/Generated_image/All/Unziped', image_name)

    # Load the image
    try:
        with Image.open(image_path) as img:
            reward = calc_alignment_score(prompt, img)
            results.append(reward)
            expected_results.append(human_feedback_score)
    except IOError:
        print(f"Failed to load image {image_name}")

# Convert results to a DataFrame for easy viewing or further processing

print(results)
print(expected_results)

[0.6143867373466492, 0.6149784326553345, 0.6160792112350464, 0.6176676154136658, 0.6087421178817749, 0.6074963212013245, 0.6083292961120605, 0.6116605997085571, 0.6210916638374329, 0.6214909553527832, 0.6146350502967834, 0.61974036693573, 0.6099618077278137, 0.6127773523330688, 0.6125150918960571, 0.6113465428352356, 0.6108604073524475, 0.6125130653381348, 0.6130713224411011, 0.6150689721107483, 0.6081223487854004, 0.6080615520477295, 0.6062483787536621, 0.6109358668327332, 0.60930997133255, 0.610578715801239, 0.6084345579147339, 0.612834632396698, 0.6088966727256775, 0.6069024801254272, 0.6091981530189514, 0.6075484156608582, 0.6088941693305969, 0.6109797358512878, 0.6095544099807739, 0.6096092462539673, 0.613615870475769, 0.6150211095809937, 0.6144526600837708, 0.6123830080032349, 0.6129132509231567, 0.6100931167602539, 0.6130741238594055, 0.6134865880012512, 0.614372968673706, 0.6122105121612549, 0.6101976633071899, 0.6118740439414978, 0.6038957834243774, 0.6023136377334595, 0.60534

In [15]:
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
import re  # Import regular expressions

# Helper function to clean and convert data
def clean_and_convert(data):
    cleaned_data = []
    for item in data:
        try:
            # Ensure item is a string and remove non-numeric characters
            cleaned_item = re.sub(r'[^0-9\.]', '', str(item))
            cleaned_data.append(float(cleaned_item))
        except ValueError:
            # Handle the case where conversion is not possible
            print(f"Skipping item {item} due to conversion error.")
    return np.array(cleaned_data, dtype=float)


# Clean and convert strings to floats
subjective_scores = clean_and_convert(expected_results)
objective_scores = np.array(results, dtype=float)

print("Sizes:", subjective_scores.size, objective_scores.size)

# Calculate PLCC
plcc, _ = pearsonr(subjective_scores, objective_scores)

# Calculate SROCC
srocc, _ = spearmanr(subjective_scores, objective_scores)

# Calculate KROCC
krocc, _ = kendalltau(subjective_scores, objective_scores)

print("PLCC:", plcc)
print("SROCC:", srocc)
print("KROCC:", krocc)


Sizes: 1600 1600
PLCC: 0.23459682621893913
SROCC: 0.2176208505927178
KROCC: 0.14638107156785746
